### Displacement Invariant Transformer (**Simple** usage example)

for more complex usage, please check the documentation: *ToDo: add documentation*

In [1]:
import os
os.chdir("..")

from nifeatures.transform import DisplacementInvariantTransformer
from nifeatures.search import TransformerCV
from nilearn.datasets import load_mni152_brain_mask
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

In [2]:
# Generate 100 simulated fMRI activation maps;
MNI = load_mni152_brain_mask(resolution=2)
template_data = MNI.get_fdata()
data = []
target = []

for sample in range(100):
    noisy_data = template_data.copy()
    weight = np.random.normal(0, 1, 1)[0]
    noisy_data[noisy_data == 1] = np.random.normal(0, 1, np.sum(template_data == 1)) * weight
    if sample == 0:
        data = noisy_data.flatten()
    else:
        data = np.vstack((data, noisy_data.flatten()))
    
    target.append(weight)

### Usage example without Hyperparameter Search

In [3]:
# The transformer is sklearn compatible.
# This means we can use it inside a sklearn pipeline:
model = Pipeline([
    ("trf", DisplacementInvariantTransformer()),
    ("model", Ridge())
])

# Fit the model;
X_train, X_test, y_train, y_test = train_test_split(data, target)
fit = model.fit(X_train, y_train)

# Predict test values and score performance;
prediction = fit.predict(X_test)
print("R2: ", np.corrcoef(prediction, y_test)[0, 1]**2)

R2:  0.013715633618947275


### Usage example with Hyperparameter Search

The transformer comes with its own Hyperparameter Search algorithm, called "TransformerCV".
The characteristic of this search algorithm is that it takes in any sklearn compatible
hyperparameter search class (e.g. GridsearchCV) and embeds it in a workflow tailored 
for the Displacement Invariant Transformer.

It should work with Scikit-Learn Hyper-parameter optimizers:
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)

and the Ray-Tune sklearn API:
https://docs.ray.io/en/latest/tune/api/sklearn.html

In [4]:
model = Pipeline([
    ("trf", DisplacementInvariantTransformer()),
    ("model", Ridge())
])

params = {
    "trf__n_peaks": [100, 200],
    "model__alpha": [0.5, 1.5]
}

# Fit the model;
X_train, X_test, y_train, y_test = train_test_split(data, target)
fit = TransformerCV(model, params, search=GridSearchCV, cv=5, n_jobs=4).fit(X_train, y_train)

# Predict test values and score performance;
prediction = fit.predict(X_test)
print("R2: ", np.corrcoef(prediction, y_test)[0, 1]**2)

R2:  0.0044949177926758765


If no search algorithm is given as a parameter, TransformerCV will return an 
array containing the results of the transformer's fit on all possible parameter
combinations (precomputed data)

In [5]:
model = Pipeline([
    ("trf", DisplacementInvariantTransformer()),
    ("model", Ridge())
])

params = {
    "trf__n_peaks": [100, 200],
    "model__alpha": [0.5, 1.5]
}

# Fit the model;
X_train, X_test, y_train, y_test = train_test_split(data, target)
fit = TransformerCV(model, params, search=None, cv=5, n_jobs=4).fit(X_train, y_train)
pd.DataFrame(fit)

,0,1,2,3
0,[n_peaks],[100],1e2bf4336622656d183797277c13ca59,"[[34, 47, 43], [54, 25, 28], [29, 64, 56], [34..."
1,[n_peaks],[100],1e2bf4336622656d183797277c13ca59,"[[34, 47, 43], [54, 25, 28], [29, 64, 56], [34..."
2,[n_peaks],[200],7f8c0d06680560c99b240e0914efd140,"[[34, 47, 43], [54, 25, 28], [29, 64, 56], [34..."
3,[n_peaks],[200],7f8c0d06680560c99b240e0914efd140,"[[34, 47, 43], [54, 25, 28], [29, 64, 56], [34..."
4,[n_peaks],[100],a558492fc1c535e7c0f416bfdf1c3cc3,"[[43, 48, 60], [36, 79, 49], [48, 65, 42], [33..."
5,[n_peaks],[100],a558492fc1c535e7c0f416bfdf1c3cc3,"[[43, 48, 60], [36, 79, 49], [48, 65, 42], [33..."
6,[n_peaks],[200],90696c8fd622370b5d1bce6f0ba782ac,"[[43, 48, 60], [36, 79, 49], [48, 65, 42], [33..."
7,[n_peaks],[200],90696c8fd622370b5d1bce6f0ba782ac,"[[43, 48, 60], [36, 79, 49], [48, 65, 42], [33..."
8,[n_peaks],[100],fb796e3765b902e8c8b2e244b656028f,"[[67, 29, 11], [21, 37, 54], [28, 43, 37], [29..."
9,[n_peaks],[100],fb796e3765b902e8c8b2e244b656028f,"[[67, 29, 11], [21, 37, 54], [28, 43, 37], [29..."


Precomputed data contains information about the coordinates of important voxels for each combination of hyperparameters. This allows us to test the performance of our model on our data in a second moment, skipping the transformer fitting step.

In [6]:
# Test performance for each set of coordinate and every combination of hyperparameters without refitting:
model = Pipeline([
    ("trf", DisplacementInvariantTransformer(precomp=fit)),
    ("model", Ridge())
])

search = GridSearchCV(
    model,
    params,
    cv=5,
    n_jobs=4
).fit(X_train, y_train)

# Predict test values and score performance;
search.predict(X_test)
print("R2: ", np.corrcoef(prediction, y_test)[0, 1]**2)

R2:  0.010873188774875343
